## Part-B: Multiclass App Review Classification: from scratch

## Part-C: Multilabel App Review Classification: from scratch

## Part-D: Multiclass App Review Classification: Finetune an MLM

## Part-E: Multilabel App Review Classification: Finetune an MLM

## Part-F: Multiclass App Review Classification: In-Context learning using causal LLM


#### Zero-shot learning

#### Few-shot learning

## Part-G: Multilabel App Review Classification: In-Context learning using causal LLM


#### Zero-shot learning

#### Few-shot learning